In [1]:
%pip install pandas requests bs4 numpy 

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from bs4 import BeautifulSoup
import csv
import requests

In [3]:
url = "https://www.byupathway.edu/policies/handbook/"
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

In [4]:
print(type(soup))

<class 'bs4.BeautifulSoup'>


In [9]:
sections = soup.select('bsp-book>ul>li>bsp-chapter>ul>li>bsp-chapter', class_='Chapter')
print(len(sections))


12


In [8]:
import pandas as pd

data = []
for section in sections:
    div = section.find('div', class_='Chapter-title')
    
    # does the div exist?
    if div:
        # if yes, is the first-child a span or an a tag?
        span = div.find('span', class_='Link')
        anchor = div.find('a', class_='Link')
        
        # name the section depending on the firstchild
        if span:
            section_name = span.text.strip()
        elif anchor:
            section_name = anchor.text.strip()
        else:
            section_name = None  
        # print(section_name)
        links = section.find_all('a', class_='Link')
        for link in links:
            title_span = link.find('span', class_='Link-span')
            title = title_span.text.strip() if title_span else ''
            url = link['href']
            data.append([section_name, title, url])


# DataFrame to CSV
df = pd.DataFrame(data, columns=['Section', 'Title', 'URL'])
df.to_csv('links.csv', index=False)


In [11]:
df = pd.read_csv('link_output.csv')
df.insert(1, 'Subsection', '')  
df.to_csv('stdhndbk.csv', index=False)